# 企业洞察

- 近 10 年全球企业 OpenRank 演变图
  - 绘制 Bump Charts
- 近 10 年中国企业 OpenRank 演变图
  - 绘制 Bump Charts
- ## 中国企业在 GitHub/Gitee 平台上的 OpenRank 占比


In [1]:
import requests
import sys

# TODO 包装这个 url 请求 主要切换 chinese / global , 切换年份 2023 需要采集 10 年的数据
#  activity 和 open_digger 的数据结构不同，需要分开处理
url = "https://xlab-open-source.oss-cn-beijing.aliyuncs.com/open_leaderboard/open_rank/company/chinese/2023.json"

response = requests.get(url)

if response.status_code == 200:
    raw_dict = response.json()
else:
    print("Error: Failed to retrieve data from the URL")
    # 不要直接 exit() 会杀掉 python 进程。
    sys.exit()

raw_dict

{'type': 'Company_China_Year',
 'time': 2023,
 'data': [{'item': {'name': 'Huawei'},
   'rank': 1,
   'value': 50903.18,
   'rankDelta': 0,
   'valueDelta': 12792.71},
  {'item': {'name': 'Alibaba'},
   'rank': 2,
   'value': 27786.2,
   'rankDelta': 0,
   'valueDelta': -1806.7},
  {'item': {'name': 'Baidu'},
   'rank': 3,
   'value': 18226.64,
   'rankDelta': 0,
   'valueDelta': -2706.52},
  {'item': {'name': 'Ant group'},
   'rank': 4,
   'value': 16823.45,
   'rankDelta': 0,
   'valueDelta': 308.2},
  {'item': {'name': 'Tencent'},
   'rank': 5,
   'value': 7542.7,
   'rankDelta': 0,
   'valueDelta': -4214.28},
  {'item': {'name': 'ByteDance'},
   'rank': 6,
   'value': 7465.38,
   'rankDelta': 2,
   'valueDelta': 1951.22},
  {'item': {'name': 'PingCAP'},
   'rank': 7,
   'value': 6618.72,
   'rankDelta': -1,
   'valueDelta': -1503.39},
  {'item': {'name': 'ESPRESSIF'},
   'rank': 8,
   'value': 6272.28,
   'rankDelta': -1,
   'valueDelta': 605.43},
  {'item': {'name': 'Fit2Cloud'},


In [3]:
from pydantic import BaseModel
from typing import List


# class ActivityEntryItem(BaseModel):
#     name: str
#     issue_comment: str
#     open_issue: str
#     open_pull: str
#     review_comment: str
#     merged_pull: str


# class ActivityEntry(BaseModel):
#     item: ActivityEntryItem
#     rank: int
#     value: float
#     rankDelta: int
#     valueDelta: float


# class Activity(BaseModel):
#     type: str
#     time: str
#     data: List[ActivityEntry]


class OpenRankEntryItem(BaseModel):
    name: str


class OpenRankEntry(BaseModel):
    item: OpenRankEntryItem
    rank: int
    value: float
    rankDelta: int
    valueDelta: float


class OpenRank(BaseModel):
    type: str
    time: int
    data: List[OpenRankEntry]


open_rank_result = OpenRank.model_validate(raw_dict)

open_rank_result.model_dump()

{'type': 'Company_China_Year',
 'time': 2023,
 'data': [{'item': {'name': 'Huawei'},
   'rank': 1,
   'value': 50903.18,
   'rankDelta': 0,
   'valueDelta': 12792.71},
  {'item': {'name': 'Alibaba'},
   'rank': 2,
   'value': 27786.2,
   'rankDelta': 0,
   'valueDelta': -1806.7},
  {'item': {'name': 'Baidu'},
   'rank': 3,
   'value': 18226.64,
   'rankDelta': 0,
   'valueDelta': -2706.52},
  {'item': {'name': 'Ant group'},
   'rank': 4,
   'value': 16823.45,
   'rankDelta': 0,
   'valueDelta': 308.2},
  {'item': {'name': 'Tencent'},
   'rank': 5,
   'value': 7542.7,
   'rankDelta': 0,
   'valueDelta': -4214.28},
  {'item': {'name': 'ByteDance'},
   'rank': 6,
   'value': 7465.38,
   'rankDelta': 2,
   'valueDelta': 1951.22},
  {'item': {'name': 'PingCAP'},
   'rank': 7,
   'value': 6618.72,
   'rankDelta': -1,
   'valueDelta': -1503.39},
  {'item': {'name': 'ESPRESSIF'},
   'rank': 8,
   'value': 6272.28,
   'rankDelta': -1,
   'valueDelta': 605.43},
  {'item': {'name': 'Fit2Cloud'},


In [ ]:
import polars as pl

table = pl.DataFrame(open_rank_result.data)

table

item,rank,value,rankDelta,valueDelta
struct[1],i64,f64,i64,f64
"{""Huawei""}",1,6432.65,0,1214.89
"{""Alibaba""}",2,2427.71,0,163.63
"{""Ant group""}",3,1747.24,0,168.7
"{""Baidu""}",4,1482.1,0,105.08
"{""ByteDance""}",5,822.39,0,48.77
"{""ESPRESSIF""}",6,633.13,1,87.0
"{""Fit2Cloud""}",7,587.08,2,79.01
"{""PingCAP""}",8,586.73,0,41.42
"{""Tencent""}",9,565.62,-3,13.07
